In [1]:
import tensorflow as tf
from keras.utils import image_dataset_from_directory
from keras.applications.efficientnet import preprocess_input

from dataset_util import split_dataset


def base_preprocess(image, label):
    image = preprocess_input(image)
    return image, label


def preprocess(image, label):
    image, label = base_preprocess(image, label)
    thr = 0.005
    label += thr
    label = tf.clip_by_value(label, clip_value_min=0, clip_value_max=1 - thr * 9)
    return image, label


def gen_dataset(img_size, batch_size):
    # split_dataset()

    # データセットの読み込み
    train_dataset = image_dataset_from_directory(
        "./Flowers/Train",
        image_size=img_size,
        batch_size=batch_size,
        label_mode="categorical",  # クラス分類
        shuffle=True,
    )
    train_dataset = train_dataset.map(preprocess)

    val_dataset = image_dataset_from_directory(
        "./Flowers/Val",
        image_size=img_size,
        batch_size=batch_size,
        label_mode="categorical",
        shuffle=False,
    )
    val_dataset = val_dataset.map(base_preprocess)
    return train_dataset, val_dataset


2025-03-11 11:20:35.395453: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-11 11:20:35.567444: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-11 11:20:36.562148: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2025-03-11 11:20:36.562263: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.applications import EfficientNetB1
from keras import layers, models
from keras.regularizers import l2

# 画像のサイズ（EfficientNetの標準サイズに合わせる）
BATCH_SIZE = 32

input_shape = (224, 224, 3)
train_dataset, val_dataset = gen_dataset(input_shape[:2], 32)

X_spec, y_spec = train_dataset.element_spec
num_classes = y_spec.shape[1:][0]

# モデルのベースを作成（事前学習済みの EfficientNetB0 を利用）
base_model = EfficientNetB1(
    weights="imagenet", include_top=False, input_shape=input_shape
)

base_model.trainable = False

model = models.Sequential(
    [
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dense(128, activation="relu", kernel_regularizer=l2(5e-4)),
        layers.Dropout(0.5),  # 過学習を防ぐためにDropoutを追加
        layers.Dense(num_classes, activation="softmax"),  # 10クラス分類
    ]
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-8)

early_stopping = EarlyStopping(
    monitor="val_loss", patience=7, verbose=1, mode="min"
)

H = model.fit(
    train_dataset,
    epochs=50,
    validation_data=val_dataset,
    callbacks=[reduce_lr, early_stopping],
)

Found 480 files belonging to 10 classes.


2025-03-11 11:20:37.966992: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-03-11 11:20:38.030126: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-03-11 11:20:38.030383: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-03-11 11:20:38.031111: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Found 120 files belonging to 10 classes.
Epoch 1/50


2025-03-11 11:20:50.262098: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8907
2025-03-11 11:20:50.299676: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2025-03-11 11:20:50.411821: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


15/15 [==============================] - 11s 219ms/step - loss: 1.8626 - accuracy: 0.5229 - val_loss: 1.1231 - val_accuracy: 0.7667 - lr: 0.0010
Epoch 2/50
15/15 [==============================] - 1s 60ms/step - loss: 0.8577 - accuracy: 0.8708 - val_loss: 0.7775 - val_accuracy: 0.8833 - lr: 0.0010
Epoch 3/50
15/15 [==============================] - 1s 59ms/step - loss: 0.7179 - accuracy: 0.9417 - val_loss: 0.6670 - val_accuracy: 0.9083 - lr: 0.0010
Epoch 4/50
15/15 [==============================] - 1s 59ms/step - loss: 0.6405 - accuracy: 0.9583 - val_loss: 0.6051 - val_accuracy: 0.9417 - lr: 0.0010
Epoch 5/50
15/15 [==============================] - 1s 56ms/step - loss: 0.6412 - accuracy: 0.9646 - val_loss: 0.5700 - val_accuracy: 0.9667 - lr: 0.0010
Epoch 6/50
15/15 [==============================] - 1s 63ms/step - loss: 0.5970 - accuracy: 0.9771 - val_loss: 0.5541 - val_accuracy: 0.9750 - lr: 0.0010
Epoch 7/50
15/15 [==============================] - 1s 60ms/step - loss: 0.5850 - acc

In [3]:
img_size = (224, 224, 3)
test_dataset = image_dataset_from_directory(
    "./Test",
    image_size=img_size[:2],
    batch_size=64,
    label_mode="categorical",
    shuffle=False,
)
test_dataset = test_dataset.map(base_preprocess)

test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test Accuracy: {test_acc:.4f}")

Found 200 files belonging to 10 classes.
4/4 [==============================] - 0s 67ms/step - loss: 0.4100 - accuracy: 0.9350
Test Accuracy: 0.9350
